<a href="https://colab.research.google.com/github/ChuMIPNSNG/-/blob/main/%E7%AC%AC%E4%BA%94%E6%AC%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets torch
!pip install transformers[torch]
!pip install accelerate -U





     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 5.4 MB/s eta 0:00:00


In [2]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torch  # Importing PyTorch

In [3]:

# Checking and setting the device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
import torch
print(torch.cuda.is_available())


Using device: cuda
True


In [4]:
from google.colab import files
import pandas as pd
import io

uploaded = files.upload()

# 确保文件名与上传的文件名相匹配
train_df = pd.read_csv(io.BytesIO(uploaded['train_dataset.csv']), encoding='gbk')


Saving train_dataset.csv to train_dataset.csv


In [5]:
# Encoding string labels to integers
label_encoder = LabelEncoder()
train_df['label'] = label_encoder.fit_transform(train_df['label'])

In [6]:
# Splitting the training data for training and validation
train_data, val_data = train_test_split(train_df, test_size=0.2)


In [7]:
# Convert DataFrames to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_data)
val_dataset = Dataset.from_pandas(val_data)


In [8]:
# Initialize the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')


tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

In [9]:
# Tokenization and label processing function
def tokenize_and_process_function(examples):
    tokenized_inputs = tokenizer(examples['query'], padding='max_length', truncation=True, max_length=128)
    tokenized_inputs['label'] = examples['label']
    return tokenized_inputs

In [10]:
# Tokenize the datasets
tokenized_train_dataset = train_dataset.map(tokenize_and_process_function, batched=True)
tokenized_val_dataset = val_dataset.map(tokenize_and_process_function, batched=True)

Map:   0%|          | 0/1668 [00:00<?, ? examples/s]

Map:   0%|          | 0/418 [00:00<?, ? examples/s]

In [11]:
# Load a pre-trained BERT model
model = BertForSequenceClassification.from_pretrained('bert-base-chinese', num_labels=len(label_encoder.classes_)).to(device)

model.safetensors:   0%|          | 0.00/412M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
# Define training arguments
!pip install transformers[torch]
!pip install accelerate -U
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',               # 结果输出目录
    num_train_epochs=200,                  # 训练周期数
    per_device_train_batch_size=8,        # 每个设备的训练批量大小
    per_device_eval_batch_size=8,         # 每个设备的评估批量大小
    warmup_steps=1000,                    # 预热步数
    weight_decay=0.01,                    # 权重衰减
    evaluation_strategy='epoch',          # 每个epoch结束时进行评估
    save_strategy='epoch',                # 设置保存策略与评估策略相同
    learning_rate=5e-5,                   # 学习率
    lr_scheduler_type='linear',           # 学习率调度器类型
    logging_dir='./logs',                 # 日志目录
    load_best_model_at_end=True,          # 训练结束时加载最佳模型
    metric_for_best_model='loss',         # 根据loss选择最佳模型
    greater_is_better=False               # 对于loss，较小的值更好
    # early_stopping_patience=3           # 早停耐心值（如果需要）
)

# 接下来，您可以使用这些参数来配置训练器



In [13]:
# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
)

In [ ]:
# Train the model
trainer.train()


Epoch,Training Loss,Validation Loss
1,No log,6.396642
2,No log,6.170306
3,6.296000,5.783130
4,6.296000,5.429754
5,5.249500,4.945886
6,5.249500,4.411440
7,5.249500,4.023911
8,3.678600,3.594760
9,3.678600,3.209395
10,2.189100,2.868226


Epoch,Training Loss,Validation Loss
1,No log,6.396642
2,No log,6.170306
3,6.296000,5.783130
4,6.296000,5.429754
5,5.249500,4.945886
6,5.249500,4.411440
7,5.249500,4.023911
8,3.678600,3.594760
9,3.678600,3.209395
10,2.189100,2.868226


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Evaluate the model
evaluation_results = trainer.evaluate()

In [ ]:
print(evaluation_results)

In [ ]:
from google.colab import files
import pandas as pd
import io


uploaded = files.upload()
# 确保文件名与上传的文件名相匹配
test_df = pd.read_csv(io.BytesIO(uploaded['test.csv']), encoding='utf-8')


In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
tokenized_test = tokenizer(list(test_df['sentence']), padding=True, truncation=True, max_length=128, return_tensors="pt")


In [ ]:
model.eval()  # 将模型设置为评估模式

# 如果使用 GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 获取预测
with torch.no_grad():
    outputs = model(**tokenized_test.to(device))
    predictions = torch.argmax(outputs.logits, dim=1).cpu().numpy()


In [ ]:

from google.colab import files
import pandas as pd
import io


uploaded = files.upload()
# 确保文件名与上传的文件名相匹配


# 读取上传的文件
submission_df = pd.read_csv(io.BytesIO(uploaded['submittion.csv']),encoding='gbk')
# 假设 predictions 包含模型的预测结果（数字）
decoded_predictions = label_encoder.inverse_transform(predictions)

# 现在 decoded_predictions 包含原始文本标签

# 将预测添加到 DataFrame 并保存为 CSV
submission_df['label'] = decoded_predictions
submission_df.to_csv('submittion.csv', index=False)

# 下载文件到本地
files.download('submittion.csv')
